In [1]:
from sqlalchemy import create_engine
import urllib
import pandas as pd
import numpy as np

## Conexion a la base de datos AdventureWorks2022

### Primero comprobaré si puedo conectarme a la base de datos SQL Server

In [5]:
SERVER = "localhost\SQLEXPRESS"
DATABASE = "AdventureWorks2022"

#Creacion de parámetros
params = urllib.parse.quote_plus(f"DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={SERVER};DATABASE={DATABASE};Trusted_Connection=yes")

#Creacion de la cadena para conectarse
connection_string = f"mssql+pyodbc:///?odbc_connect={params}"

#Inicialización de un objeto conexión
conn = create_engine(connection_string)

## 1. Extrae los datos de los clientes de la tabla Sales.Customer junto a la información de cada venta de la tabla Sales.SalesOrderHeader

In [8]:
#df_customers = pd.read_sql("SELECT c.CustomerID, salesOrd.OrderDate, salesOrd.TotalDue FROM Sales.Customer AS c JOIN Sales.SalesOrderHeader AS salesOrd ON c.CustomerID = salesOrd.CustomerID", conn)

df_customers = pd.read_sql("SELECT c.CustomerID FROM Sales.Customer AS c", conn)
df_salesOrd = pd.read_sql("SELECT CustomerID, OrderDate, TotalDue FROM Sales.SalesOrderHeader", conn)

df_joins = df_customers.merge(df_salesOrd, on="CustomerID", sort=True)

print(df_joins.head(10))

   CustomerID  OrderDate   TotalDue
0       11000 2011-06-21  3756.9890
1       11000 2013-06-20  2587.8769
2       11000 2013-10-03  2770.2682
3       11001 2011-06-17  3729.3640
4       11001 2013-06-18  2674.0227
5       11001 2014-05-12   650.8008
6       11002 2011-06-09  3756.9890
7       11002 2013-06-02  2535.9640
8       11002 2013-07-26  2673.0613
9       11003 2011-05-31  3756.9890


# 2.  Utiliza group by para obtener las siguientes columnas en el dataframe:
· 2.1- Total vendido al cliente
· 2.2- Fecha del primer pedido del cliente
· 2.3- Fecha del último pedido del cliente
· 2.4- Número de pedidos del cliente

In [11]:
df_joins_agg = df_joins.groupby("CustomerID").agg(
    total_vendido=pd.NamedAgg("TotalDue", aggfunc="sum"),
    primer_pedido=pd.NamedAgg("OrderDate", aggfunc="min"),
    ultimo_pedido=pd.NamedAgg("OrderDate", aggfunc="max"),
    tot_pedidos=pd.NamedAgg("OrderDate", aggfunc="count")
)

print(df_joins_agg)

            total_vendido primer_pedido ultimo_pedido  tot_pedidos
CustomerID                                                        
11000           9115.1341    2011-06-21    2013-10-03            3
11001           7054.1875    2011-06-17    2014-05-12            3
11002           8966.0143    2011-06-09    2013-07-26            3
11003           8993.9155    2011-05-31    2013-10-10            3
11004           9056.5911    2011-06-25    2013-10-01            3
...                   ...           ...           ...          ...
30114          13135.0333    2012-05-30    2014-03-01            8
30115          10033.8172    2012-06-30    2014-03-31            8
30116         211671.2674    2013-05-30    2014-03-01            4
30117         919801.8188    2011-07-01    2014-03-31           12
30118         313671.5352    2012-07-31    2014-05-01            8

[19119 rows x 4 columns]


## 3.  Añade las siguientes columnas en el dataframe:
- 3.1- Tiempo de vida del cliente (Esto será el tiempo transcurrido desde el primer pedido hasta el último pedido)
- 3.2- Media del dinero gastado por venta del cliente(Será el gasto total del cliente)
- 3.3- Valor del Tiempo de Vida del Cliente o CLV(CLV=Valor Promedio de Compra×Número de Compras×(Tiempo de Vida del Cliente(días)/365)


In [14]:
df_joins_agg["tiempo_v_cliente"] = (df_joins_agg["ultimo_pedido"] - df_joins_agg["primer_pedido"]).dt.days
df_joins_agg["media_dinero_gastado"] = df_joins_agg["total_vendido"] / df_joins_agg["tot_pedidos"]
df_joins_agg["CLV_cliente"] = (df_joins_agg["media_dinero_gastado"] * df_joins_agg["tot_pedidos"] * (df_joins_agg["tiempo_v_cliente"] / 365)).round(2)

print(df_joins_agg)

            total_vendido primer_pedido ultimo_pedido  tot_pedidos  \
CustomerID                                                           
11000           9115.1341    2011-06-21    2013-10-03            3   
11001           7054.1875    2011-06-17    2014-05-12            3   
11002           8966.0143    2011-06-09    2013-07-26            3   
11003           8993.9155    2011-05-31    2013-10-10            3   
11004           9056.5911    2011-06-25    2013-10-01            3   
...                   ...           ...           ...          ...   
30114          13135.0333    2012-05-30    2014-03-01            8   
30115          10033.8172    2012-06-30    2014-03-31            8   
30116         211671.2674    2013-05-30    2014-03-01            4   
30117         919801.8188    2011-07-01    2014-03-31           12   
30118         313671.5352    2012-07-31    2014-05-01            8   

            tiempo_v_cliente  media_dinero_gastado  CLV_cliente  
CustomerID             

## 4.Carga el dataframe resultante en una tabla llamada CustomerLifetimeValue

In [17]:
df_joins_agg.to_sql("CustomerLifetimeValue", conn, if_exists="replace")

255

## 1. Extrae los datos de los clientes de la tabla Sales.Customer junto a la información de ventas de la tabla Sales.SalesOrderHeader y detalles de pedidos de la tabla Sales.SalesOrderDetails

In [40]:
'''
df_customers_sales = pd.read_sql("""
    SELECT c.CustomerID, soh.OrderDate, soh.TotalDue, sd.ProductID, sd.OrderQty, sd.UnitPrice, sd.UnitPriceDiscount, sd.LineTotal
    FROM Sales.Customer c
        JOIN Sales.SalesOrderHeader soh ON soh.CustomerID = c.CustomerID
        JOIN Sales.SalesOrderDetail sd ON soh.SalesOrderID = sd.SalesOrderID
""", conn)
'''
df_customers = pd.read_sql("SELECT CustomerID FROM Sales.Customer", conn)
df_soh = pd.read_sql("SELECT CustomerID, SalesOrderID, OrderDate, TotalDue FROM Sales.SalesOrderHeader", conn) #salesOrderHeader
df_sod = pd.read_sql("SELECT SalesOrderID, OrderQty, UnitPrice, UnitPriceDiscount, LineTotal FROM Sales.SalesOrderDetail", conn) #salesOrderDetail

df_customer_salesheader = df_customers.merge(df_soh, on="CustomerID", how="inner")
df_customers_sales = df_customer_salesheader.merge(df_customer_salesheader, on="SalesOrderID", how="inner")

print(df_customers_sales.head(5))

   CustomerID_x  SalesOrderID OrderDate_x  TotalDue_x  CustomerID_y  \
0         11012         54508  2013-08-16     82.8529         11012   
1         11012         68413  2014-03-17      6.9394         11012   
2         11013         56137  2013-09-13     43.0729         11013   
3         11013         74908  2014-06-23     82.8529         11013   
4         11014         54898  2013-08-23     43.0729         11014   

  OrderDate_y  TotalDue_y  
0  2013-08-16     82.8529  
1  2014-03-17      6.9394  
2  2013-09-13     43.0729  
3  2014-06-23     82.8529  
4  2013-08-23     43.0729  


## 2. Utiliza group by para obtener las siguientes columnas en el dataframe:
2.1- Total de unidades vendidas de un producto en cada mes de existencia del producto
2.2- Total vendido de un producto cada mes

In [85]:
df_ctm_sales = df_customers_sales.copy(deep=True)

#Primero, agregar una columna con solo la info. del mes para cada OrderDate (OrderDate_x)
df_ctm_sales["mes"] = df_ctm_sales["OrderDate_x"].dt.month

print(df_ctm_sales.head(10))

#Ahora, usar groupby para agrupar por 
df_ctm_sales_groupby = df_ctm_sales.groupby("mes").agg(
    
    sum_sales=pd.NamedAgg("SalesOrderID", aggfunc="count"),
    totalDue_month=pd.NamedAgg("TotalDue_x", aggfunc="sum")
)

print(df_ctm_sales_groupby.head(20))

print()
print(df_ctm_sales_groupby.query("`mes` == 5"))

   CustomerID_x  SalesOrderID OrderDate_x  TotalDue_x  CustomerID_y  \
0         11012         54508  2013-08-16     82.8529         11012   
1         11012         68413  2014-03-17      6.9394         11012   
2         11013         56137  2013-09-13     43.0729         11013   
3         11013         74908  2014-06-23     82.8529         11013   
4         11014         54898  2013-08-23     43.0729         11014   
5         11014         57222  2013-09-30    109.9144         11014   
6         11021         51610  2013-06-25   2621.0158         11021   
7         11022         51556  2013-06-22   2566.1194         11022   
8         11040         55599  2013-09-03   2597.8108         11040   
9         11062         51347  2013-06-09   2598.9158         11062   

  OrderDate_y  TotalDue_y  mes  
0  2013-08-16     82.8529    8  
1  2014-03-17      6.9394    3  
2  2013-09-13     43.0729    9  
3  2014-06-23     82.8529    6  
4  2013-08-23     43.0729    8  
5  2013-09-30    109